## 카드 승인 내역 엑셀 처리
1단계로 카드승인내역에서 불필요한 라인, 불필요한 컬럼(가뱅점번호, 단말기번호) 삭제하기

In [21]:

# KICC 신용거래내역 엑셀 파일 테이터 처리

import pandas as pnds
import pickle
import datetime

# 1. 엑셀 파일을 읽어서 DataFrame 생성
filename = './downdata/20220630/신용거래내역조회 (11).xlsx'                 # KICC 신용거래내역 엑셀파일
card_history_df = pnds.read_excel(
                                filename, header=0, thousands = ',',           # 첫번쨰 라인은 컬럼명(header), 금액 천단위 구분자 ',' 고려
                                dtype={'거래고유번호': str,                         # 너무 큰 숫자들이므로 문자열로 셋팅
                                        '금액': 'int64',
                                        '승인번호': str}                            # 영문이나 숫자 '0'도 표시되어야 함으로 문자열 타입
                                )
# 2. 데이터 전처리
# 2-1. '거래일시' 컬럼명 수정
card_history_df.rename(columns={'거래일시▼': 'date'}, inplace=True)     # '거래일시' 컬럼명 'date'로 수정 => 다른 dataframe과 통일

# # 데이터 타입 변경
# card_history_df['금액'] = card_history_df['금액'].astype('int64', errors='ignore')      # '금액' : int64

# '거래고유번호' 컬럼에 있는 내용중에 NaN인 행 제거
card_history_df.dropna(subset=['거래고유번호'], inplace=True)

# 2-2. '승인번호' 중복 라인제거를 위한 중복 정보 추가
# dup_series = card_history_df.duplicated(['승인번호'], keep=False)           # '승인번호' 중복 여부 Bool series 리턴, 중복되면 모두 True
# card_history_df = pnds.concat([card_history_df, dup_series], axis=1)        # 결과를 dataframe column 으로 추가
# card_history_df.rename(columns={0: 'dup'}, inplace=True)                    # 추가된 중복 컬럼이름을 'dup'로 변경

card_history_df.drop_duplicates(subset=['승인번호'], keep=False, inplace=True)

# # 2-3. 중복 라인들을 제외하면서, 동시에 필요 컬럼들만 추출
# card_history_df = card_history_df.loc[card_history_df['dup'] == False][['거래고유번호', '승인구분', 'date', '카드번호', '발급카드사', '매입카드사', '금액']]
card_history_df = card_history_df[['거래고유번호', '승인구분', 'date', '승인번호', '카드번호', '발급카드사', '매입카드사', '금액']]

# 2-4. date 기준 Sorting
card_history_df.sort_values(by=['date'], inplace=True)

card_history_df

# # 2-5. date 포맷 변경: '%Y-%m-%d'
# card_history_df['date'] = card_history_df['date'].map(lambda str_data: str_data.split()[0], na_action='ignore') # 날짜 포맷 : '%Y-%m-%d'
# # 다른 방식으로서, 데이터타입을 datetime으로하고 date포맷을 변경하는 방법
# # opera_df['TRX_DATE'] = opera_df['TRX_DATE'].astype('datetime64[ns]', errors='ignore')    # date 포맷 변경을 위해서 datetimee 형으로 변환
# # opera_df['TRX_DATE'] = opera_df['TRX_DATE'].dt.strftime('%Y-%m-%d')     # 연산을 통해 포맷 변경 => 반환 타입은 일반 객체로 변경됨

# # card_history_df

# # DataFrame 저장
# target_date = (datetime.datetime.now() - datetime.timedelta(1)).strftime('%Y%m%d')  # 어제날짜의 폴더 이름
# base_dir = 'C:\\work\\pycard\\dtdata\\' + target_date
# now_date = card_history_df.iloc[1]['date']      # 정렬된 상태에서의 첫번쨰 인덱스를 선택 (정렬 이전의 지료에서 1번 인덱스는 다음날의 자료일 가능성이 있음)
# file_name = '/dt_kicc_history_' + now_date
# data_dir = base_dir + file_name
# with open(data_dir, "wb") as file:
#     pickle.dump(card_history_df, file)

c:\ProgramData\Anaconda3\envs\main\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,거래고유번호,승인구분,date,승인번호,카드번호,발급카드사,매입카드사,금액
32,300033181069,승인,2022-06-30 00:27:04,46944196,5371-0200-1273-8835,우리체크,비씨카드,50787
31,300233208989,승인,2022-06-30 02:02:39,678861,5137-9680-0137-4589,해외마스타,하나구외환,154000
30,300211686458,승인,2022-06-30 02:05:17,957241,5395-9387-3124-6248,해외마스타,하나구외환,49368
29,300221688148,승인,2022-06-30 02:07:41,094653,5567-1717-6091-7065,해외마스타,하나구외환,83160
28,300921395758,승인,2022-06-30 09:22:18,10606077,4015-8400-0095-6106,삼성비자,삼성카드,78166
27,300933522200,승인,2022-06-30 09:31:08,00077209,5588-1300-0406-5003,현대마스터개인,현대카드,168300
26,300911464658,승인,2022-06-30 09:33:40,29037557,5365-1016-7060-8375,카카오뱅크체크,국민카드,147730
25,300933536828,승인,2022-06-30 09:36:51,00661491,4890-1680-6625-9805,현대비자개인,현대카드,74800
23,300921520941,승인,2022-06-30 09:42:41,00703119,4890-1680-6625-9805,현대비자개인,현대카드,74800
22,300933563282,승인,2022-06-30 09:47:04,253961,5271-7310-0049-3461,해외마스타,하나구외환,186534
